# Case Law API

In [1]:
import requests
import json
from bs4 import BeautifulSoup

# Case Categorization
The `categorize_cases` function puts cases in the dictionary holder.

In [2]:
def removeHTML(s):
    html_str = BeautifulSoup(s, features='html.parser')
    return html_str.get_text()

#formats dictionary to categorize by state
def categorize_cases(all_cases, results):
    total_cases = 0
    for case in results:
        total_cases += 1
        casebody = case['casebody']['data']
        casebody = removeHTML(casebody) if not casebody is None else ''
        info = (case['name'], case['id'], casebody, case['frontend_url'])
        all_cases.append(info)
    return total_cases

# How to collect the cases from the API
The code block below has been set up for collecting whitelisted Illinois information.

In [3]:
#querying data from API

min_date = '&decision_date_min=2000-01-01'
data = {'next': 'https://api.case.law/v1/cases/?jurisdiction=ill&full_case=true&body_format=html&page_size=100&ordering=-decision_date'}
data['next'] += min_date
all_cases = []
total_cases = 0
true_total = 0
iter_count = 0
while 'next' in data:
    r = requests.get(data['next'])
    data = r.json()
    true_total = data['count']
    total_cases += categorize_cases(all_cases, data['results'])
    iter_count += 1
    # This limits to 5 * 100, adjust as needed
    if iter_count >= 5:
        break

# Resulting File
The resulting file will be placed in the same directory as this notebook; make sure it's not overwritten.

In [4]:
#dumping dict to static JSON
with open('./tester_static.json', 'w') as json_file:
    json.dump(all_cases, json_file)